- 자유도는 좀 떨어지지만 구현이 굉장히 간단함

In [1]:
# !touch finetune_plm_hftrainer.py

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# import argparse
# import random

# from sklearn.metrics import accuracy_score

# import torch

# from transformers import BertTokenizerFast
# from transformers import BertForSequenceClassification, AlbertForSequenceClassification
# from transformers import Trainer
# from transformers import TrainingArguments

# import sys
# sys.path.append('/content/drive/MyDrive/인공지능/텍스트분류')
# from simple_ntc.bert_dataset import TextClassificationCollator
# from simple_ntc.bert_dataset import TextClassificationDataset
# from simple_ntc.utils import read_text


# def define_argparser():
#     p = argparse.ArgumentParser()

#     p.add_argument('--model_fn', required=True)
#     p.add_argument('--train_fn', required=True)
#     # Recommended model list:
#     # - kykim/bert-kor-base
#     # - kykim/albert-kor-base
#     # - beomi/kcbert-base
#     # - beomi/kcbert-large
#     p.add_argument('--pretrained_model_name', type=str, default='beomi/kcbert-base')
#     p.add_argument('--use_albert', action='store_true')

#     p.add_argument('--valid_ratio', type=float, default=.2)
#     p.add_argument('--batch_size_per_device', type=int, default=32)
#     p.add_argument('--n_epochs', type=int, default=5)

#     p.add_argument('--warmup_ratio', type=float, default=.2)

#     p.add_argument('--max_length', type=int, default=100)

#     config = p.parse_args()

#     return config


# def get_datasets(fn, valid_ratio=.2):
#      # Get list of labels and list of texts.
#     labels, texts = read_text(fn)

#     # Generate label to index map.
#     unique_labels = list(set(labels))
#     label_to_index = {}
#     index_to_label = {}
#     for i, label in enumerate(unique_labels):
#         label_to_index[label] = i
#         index_to_label[i] = label

#     # Convert label text to integer value.
#     labels = list(map(label_to_index.get, labels))

#     # Shuffle before split into train and validation set.
#     shuffled = list(zip(texts, labels))
#     random.shuffle(shuffled)
#     texts = [e[0] for e in shuffled]
#     labels = [e[1] for e in shuffled]
#     idx = int(len(texts) * (1 - valid_ratio))

#     train_dataset = TextClassificationDataset(texts[:idx], labels[:idx])
#     valid_dataset = TextClassificationDataset(texts[idx:], labels[idx:])

#     return train_dataset, valid_dataset, index_to_label


# def main(config):
#     # Get pretrained tokenizer.
#     tokenizer = BertTokenizerFast.from_pretrained(config.pretrained_model_name)
#     # Get datasets and index to label map.
#     train_dataset, valid_dataset, index_to_label = get_datasets(
#         config.train_fn,
#         valid_ratio=config.valid_ratio
#     )

#     print(
#         '|train| =', len(train_dataset),
#         '|valid| =', len(valid_dataset),
#     )

#     total_batch_size = config.batch_size_per_device * torch.cuda.device_count() 
#     n_total_iterations = int(len(train_dataset) / total_batch_size * config.n_epochs) #샘플수/전체배치사이즈*n_epoch
#     n_warmup_steps = int(n_total_iterations * config.warmup_ratio)
#     print(
#         '#total_iters =', n_total_iterations,
#         '#warmup_iters =', n_warmup_steps,
#     )

#     # Get pretrained model with specified softmax layer.
#     model_loader = AlbertForSequenceClassification if config.use_albert else BertForSequenceClassification
#     model = model_loader.from_pretrained(
#         config.pretrained_model_name,
#         num_labels=len(index_to_label)
#     )

#     training_args = TrainingArguments(
#         output_dir='/content/drive/MyDrive/인공지능/텍스트분류/hf_checkpoints',
#         num_train_epochs=config.n_epochs,
#         per_device_train_batch_size=config.batch_size_per_device,
#         per_device_eval_batch_size=config.batch_size_per_device,
#         warmup_steps=n_warmup_steps,
#         weight_decay=0.01,
#         fp16=True,# 기본이 32bit float이다. 16으로 하게되면 메모리도 적게 쓰고 2000번째 gpu부터는 속도가 훨씬 빠르다.
#         evaluation_strategy='epoch',
#         logging_steps=n_total_iterations // 100,
#         save_steps=n_total_iterations // config.n_epochs,
#         load_best_model_at_end=True,
#     )

#     def compute_metrics(pred):
#         labels = pred.label_ids
#         preds = pred.predictions.argmax(-1) # 가장 확률이 높음 값의 인덱스를 가져와라

#         return {
#             'accuracy': accuracy_score(labels, preds)
#         }

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         data_collator=TextClassificationCollator(tokenizer,
#                                        config.max_length,
#                                        with_text=False),
#         train_dataset=train_dataset, # 기존은 데이터로더를 넘겨줬는데 지금은 데이터로드가 안에 있는 것이다.
#         eval_dataset=valid_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()

#     torch.save({
#         'rnn': None,
#         'cnn': None,
#         'bert': trainer.model.state_dict(),
#         'config': config,
#         'vocab': None,
#         'classes': index_to_label,
#         'tokenizer': tokenizer,
#     }, config.model_fn)


# if __name__ == '__main__': # 진입 지점
#     config = define_argparser()
#     main(config)

## train
- 왜인지는 모르겠는제 오류가 생김

In [ ]:
!python /content/drive/MyDrive/인공지능/텍스트분류/finetune_plm_hftrainer.py --model_fn /content/drive/MyDrive/인공지능/텍스트분류/models/review.hft.bert.pth --train_fn /content/drive/MyDrive/인공지능/텍스트분류/data/review.sorted.uniq.refined.shuf.train.tsv --batch_size_per_device 96 --n_epochs 2

## 결과 확인